In [6]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames

In [66]:
#Part d
function stable_regression(X, y, k)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    n, p = size(X)
    
    @variable(m, B[1:p])
    @variable(m, u[1:n])
    @variable(m, θ)
    println(typeof(B))
    println(typeof(u))
    println(typeof(y))
    println(typeof(X))
    @constraint(m, u .+ θ .>= (y - X*B).^2)
    @constraint(m, u .>= 0)
    @objective(m, Min, k*θ + sum(u))

    optimize!(m)
    return value.(B)
end;

In [62]:
data = CSV.File("final_data.csv", header=true) |> Tables.matrix;


In [63]:
y_solar = data[:,11];
X = data[:,13:38];
y_wind = data[:,12];


In [55]:
n,p = size(X)

(8759, 26)

In [35]:
k = [4380,4818,5255,5693,6131,6569,7007,7445,7883,8321]

MSE_best, λ_best = Inf, Inf
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883,8321]
    X_train = Matrix(X[1:k,:])
    y_train = y_solar[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_solar[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    # MSE_temp = MSE(y_valid, X_valid*β_temp)
    # if MSE_temp < MSE_best
    #     MSE_best = MSE_temp
    #     k_best = k 
    # end
end


In [30]:
function MSE(y, pred) 
    return sum((y-pred).^2)/length(y)
end;

In [67]:
stable_regression(X,y_solar, 8000)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Vector{VariableRef}
Vector{VariableRef}
Vector{Any}
Matrix{Any}


LoadError: MethodError: no method matching *(::String, ::VariableRef)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *(::Union{AbstractChar, AbstractString}, [91m::Union{AbstractChar, AbstractString}...[39m) at strings/basic.jl:260
[0m  *(::Union{AbstractChar, AbstractString}, [91m::Missing[39m) at missing.jl:183
[0m  ...

In [68]:
X

8759×26 Matrix{Any}:
 32.4  29.7  24.0  70.72  0.0  0  0.0   0.39  …  0  0  1  0  0  6  1  0  0  0
 32.5  32.5  24.1  70.76  0.0  0  0.0   0.34     0  1  0  0  0  6  1  0  0  0
 31.6  28.3  24.2  73.84  0.0  0  0.0   0.3      0  1  0  0  0  6  1  0  0  0
 31.6  26.8  25.1  76.66  0.0  0  0.0   0.25     0  1  0  0  0  6  1  0  0  0
 31.8  27.2  25.2  76.31  0.0  0  0.0   0.2      0  1  0  0  0  6  1  0  0  0
 29.5  23.6  24.1  79.99  0.0  0  0.0   0.15  …  0  0  1  0  0  6  1  0  0  0
 29.7  23.9  23.2  76.63  0.0  0  0.0   0.1      0  0  1  0  0  6  1  0  0  0
 28.9  22.9  21.3  72.8   0.0  0  0.0   0.05     0  0  1  0  0  6  1  0  0  0
 28.1  24.7  21.0  74.55  0.0  0  0.0   0.0      0  0  1  0  0  7  1  0  0  0
 28.8  22.7  21.2  72.71  0.0  0  2.35  2.35     0  0  0  0  1  7  1  0  0  0
 30.7  23.1  22.1  69.99  0.0  0  0.0   2.33  …  0  0  1  0  0  7  1  0  0  0
 31.9  24.0  22.1  66.79  0.0  0  0.0   2.31     0  1  0  0  0  7  1  0  0  0
 33.0  23.8  21.3  61.67  0.0  0  0.0   2.3